In [1]:
import numpy as np
import tensorflow as tf
import json

np.set_printoptions(suppress=True, precision=4, linewidth=250)
opts = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
conf = tf.ConfigProto(gpu_options=opts)
tf.enable_eager_execution(config=conf)

In [3]:
from google.protobuf.json_format import MessageToDict, ParseDict

In [ ]:
filename = "./data/bair/test/traj_0_to_255.tfrecords"
example = next(tf.python_io.tf_record_iterator(filename))
dict_message = MessageToDict(tf.train.Example.FromString(example))
print(dict_message['features']['feature'].keys())

In [6]:
filename = "/home/pmitrano/catkin_ws/src/link_bot/link_bot_data/data/moving_block/train/traj_.tfrecords"
example = next(tf.python_io.tf_record_iterator(filename))
dict_message = MessageToDict(tf.train.Example.FromString(example))
print(dict_message['features']['feature'].keys())

NotFoundError: /home/pmitrano/catkin_ws/src/link_bot/link_bot_data/data/moving_block/train/traj_0_to_127.tfrecords; No such file or directory

In [ ]:
ls data